In [ ]:
pip install ISLP

# Classification

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize)

from ISLP import confusion_table
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression as LR
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.naive_bayes import GaussianNB as NB
from sklearn.neighbors import KNeighborsClassifier as KNN

from ISLP.models import (ModelSpec as MS, summarize)
import statsmodels.api as sm

## Load data set

In [ ]:
data = load_data('Smarket')
data.head()
# data.shape

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
1,2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2,2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
3,2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
4,2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


## Logistic Regression


In [ ]:
predictors = data.columns.drop(['Year','Volume','Today' , 'Direction'])
design = MS(predictors)
X = design.fit_transform(data) # intercept values
y = data.Direction == 'Up'
model1 = sm.GLM(y , X , family=sm.families.Binomial())
result = model1.fit() # train the model
summarize(result)

,coef,std err,z,P>|z|
intercept,0.0742,0.057,1.309,0.191
Lag1,-0.0713,0.050,-1.424,0.155
Lag2,-0.0441,0.050,-0.882,0.378
Lag3,0.0092,0.050,0.185,0.853
Lag4,0.0072,0.050,0.145,0.885
Lag5,0.0093,0.049,0.188,0.851


Logitic regression testing

In [ ]:
predictions = result.predict() # predict using model
labels = np.array(['Down'] * len(data))
labels[predictions > 0.5] = 'Up'
confusion_table(labels , data.Direction)

Truth,Down,Up
Predicted,,
Down,116,98
Up,486,550


In [ ]:
accuracy_for_logistic = (116+550)/(116+550+98+486)
accuracy_for_logistic

0.5328

## Linear Disc Analysis (LDA)

In [ ]:
model2 = LDA() # create a Model
X = X.drop(['intercept'] , axis=1) # delete intercept col


In [ ]:
model2.fit(X,y) # train the model

LinearDiscriminantAnalysis()

In [ ]:
lda_predictions = model2.predict(X)
# lda_predictions
# np.all(np.where(lda_pred[:, 1] >= 0.5 , 'Up' , 'Down') = lda_predictions)
confusion_table(lda_predictions , y)

Truth,False,True
Predicted,,
False,113,98
True,489,550


In [ ]:
accuracy_for_LDA = (113 + 550)/(113 + 98 + 489 + 550)
accuracy_for_LDA

0.5304

## Naive Bayes Classifier

In [ ]:
model_nb = NB()
model_nb.fit(X,y)
nb_predict = model_nb.predict(X)
confusion_table(nb_predict , y)

Truth,False,True
Predicted,,
False,129,101
True,473,547


In [ ]:
(129 + 547) / (129 + 101 + 473 + 547) #accuracy

0.5408

## KNN

In [ ]:
knn_model = KNN(n_neighbors=3)
knn_model.fit(X,y)
knn_pred = knn_model.predict(X)
confusion_table(knn_pred , y)

Truth,False,True
Predicted,,
False,441,151
True,161,497


In [ ]:
(411 + 497)/(411 + 497 + 151 + 151) #accuracy

0.7504132231404959